# LiCE: Likely Counterfactual Explanations
[LiCE](https://openreview.net/pdf?id=rGyi8NNqB0) is a method of finding high-quality Counterfactual Explanations. LiCE utilizes a Sum-Product Network, a probabilistic model, to estimate the plausibility (i.e., likelihood) of the counterfactual sample. Through the use of Mixed-Integer Optimization, we are able to find globally optimal (i.e., closest/most likely) counterfactuals, while satisfying various constraints on the data. These can be related to actionability (a feature cannot change or can only increase/decrease), causality (change of one attribute requires change in another) or data type (categorical, ordinal, discrete and real data).

In this notebook, we showcase the use of the algorithm with all its parameters.

We evaluate on [Give me some credit (GMSC)](https://www.kaggle.com/c/GiveMeSomeCredit/data) data.

In [ ]:
import sys
import os

# Go one directory up to the root (from examples/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.insert(0, project_root)

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from humancompatible.explain.lice.data import DataHandler, Monotonicity
from humancompatible.explain.lice.spn import SPN
from humancompatible.explain.lice import LiCE

# just a small wrapper of a PyTorch model, we export it to onnx later
from nn_model import NNModel

## Prepare data and models
To find counterfactuals, we first need to describe the data constraints. Then we train a Neural Network which we will be explaining and a Sum-Product Network which we will use to estimate plausibility.

### Setup data context (mutability, feature types...)
We load the data and set up the `DataHandler` with all data constraints.

In [2]:
data = pd.read_csv(project_root+"/data/GMSC.csv", index_col=0).dropna()
X = data[data.columns[1:]]
y = data[["SeriousDlqin2yrs"]]

# set bounds on feature values
# either fixed by domain knowledge
bounds = {"RevolvingUtilizationOfUnsecuredLines": (0, 1), "DebtRatio": (0, 2)}
# or take them from the data
for col in X.columns:
    if col not in bounds:
        bounds[col] = (X[col].min(), X[col].max())

config = {
    "categ_map": {}, # categorical features, map from feature names to a list of categ values, e.g. {"sex": ["male", "female"] | if one provides an empty list with a feature name, then all unique values are taken as categories - note that training split does not have to include all values...
    "ordered": [], # list of featurenames that contain ordered categorical values, e.g. education levels
    "discrete": [
        "NumberOfTime30-59DaysPastDueNotWorse",
        "age",
        "NumberOfOpenCreditLinesAndLoans",
        "NumberOfTimes90DaysLate",
        "NumberRealEstateLoansOrLines",
        "NumberOfTime60-89DaysPastDueNotWorse",
        "NumberOfDependents",
    ], # contiguous discrete fearures
    "bounds_map": bounds, # bounds on all contiguous values
    "immutable": ["NumberOfDependents"], # features that cannot change
    "monotonicity": {"age": Monotonicity.INCREASING}, # features that can only increase (or decrease)
    "causal_inc": [], # causal increase, pairs of features where if one increases, the other one must increase as well, e.g. [("education", "age")]
    "greater_than": [], # inter-feature dependence, one feature must always be > anohter feature, a list of pairs, e.g. [("# total missed payments",  "# missed payments last year")]
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
dhandler = DataHandler(
    X_train,
    y_train,
    **config,
    # optionally, one can provide the list of feature names (and target name) but here we pass pandas dataframe (and a series) with named columns, which will be taken as feature names
)

# finally, encode the input data
X_train_enc = dhandler.encode(X_train, normalize=True, one_hot=True)
y_train_enc = dhandler.encode_y(y_train, one_hot=False)

### Train a Neural Network

LiCE works with `onnx` files, here we use PyTorch to train and export an example network. Any ReLU network exported to `onnx` will work. Last layer should not contain the sigmoid activation when passed to LiCE.

In [3]:
nn = NNModel(dhandler.encoding_width(True), [20, 10], 1)
nn.train(X_train_enc, y_train_enc)
# output it to ONNX file
nn.save_onnx("tmp_nn.onnx")

Training:


100%|███████████████████████████████████████████████████████████████████████████████████| 50/50 [02:32<00:00,  3.04s/it]


### Train an SPN
The Sum-Product Network will allow us to evaluate the likelihood of the Counterfactual. This allows us to find _plausible_ counterfactals.

In [4]:
# this can take long...
# setting "min_instances_slice":1000 argument leads to faster training by allowing leaves to be formed on >=1000 samples (default is 200)

# data should be numpy array of original (non-encoded) values, should include the target as last feature
spn_data = np.concatenate([X_train.values, y_train.values], axis=1)
spn = SPN(spn_data, dhandler, normalize_data=False, learn_mspn_kwargs={"min_instances_slice":1000})

## Using LiCE

In [5]:
top_n = 1 # top-n counterfactuals to look for (obtaining more than 1 is implemented only for Gurobi solver)
time_limit = 120 # number of seconds to look for a counterfactual, after which the best solution so far is returned
# solver to choose
solver_name = "gurobi" # Gurobi solver is a proprietary solver used in the original experiments. Academic licenses are available.
# Use e.g. HiGHS if you do not have access to gurobi solver. Install with: $ pip install highspy
# solver_name = "appsi_highs"

lice = LiCE(
    spn,
    nn_path="tmp_nn.onnx",
    data_handler=dhandler,
)

# take a sample
np.random.seed(1)
i = np.random.randint(X_test.shape[0])

sample = X_test.iloc[i]
enc_sample = dhandler.encode(X_test.iloc[[i]])
# we assume binary classification without sigmoid activation in the last layer
prediction = nn.predict(enc_sample) > 0
if prediction[0][0]:
    print("The sample is classified as having a 90 day past due delinquency.")
else:
    print("The sample is classified as not having a 90 day past due delinquency.")

The sample is classified as not having a 90 day past due delinquency.


### Thresholding variant
First, we showcase the version where we find the closest counterfactual explanation, such that its **likelihood is above a given threshold**. In this case, we choose the threshold as 25th percentile of the training data.

If you get a `GurobiError` stating that model is too big, this means that you should either obtain a license for gurobi or use a different solver (e.g. HiGHS). To set this up, check out the previous cell.

In [6]:
lls = spn.compute_ll(spn_data)
quartile_ll = np.quantile(lls, 0.25) # threhsold on CE likelihood

thresholded = lice.generate_counterfactual(
    sample,
    not prediction,
    ll_threshold=quartile_ll,
    n_counterfactuals=top_n,
    time_limit=time_limit,
    solver_name=solver_name
)

if lice.stats["optimal"]:
    print("A globally optimal solution found!")

A globally optimal solution found!


In [7]:
print("Suggested counterfactual changes:")
for n, val in zip(sample.index, thresholded[0] - sample.values):
    if val > 0:
        print(f"  Increase the {n} parameter by {val}")
    elif val < 0:
        print(f"  Decrease the {n} parameter by {-val}")

Suggested counterfactual changes:
  Increase the RevolvingUtilizationOfUnsecuredLines parameter by 0.22284848599999996
  Increase the NumberOfTime30-59DaysPastDueNotWorse parameter by 1.0
  Decrease the DebtRatio parameter by 0.092052859
  Decrease the MonthlyIncome parameter by 89.56047399999989
  Increase the NumberOfTimes90DaysLate parameter by 3.0
  Decrease the NumberOfTime60-89DaysPastDueNotWorse parameter by 1.0


### Optimizing variant
An aternative to the thresholding variant, here we minimize a linear combination of distance to factual and the likelihood. Set the `alpha` parameter higher if you want to stress the likelihood more, or lower if you want to focus more on proximity.

Taking `alpha` approximately equal to (mean CE distance)/(mean CE log-likelihood) works quite well in balancing the two goals.

In [8]:
alpha = 0.1 # coefficient of the linear combination in the objective

optimized = lice.generate_counterfactual(
    sample,
    not prediction,
    ll_opt_coefficient=alpha,
    n_counterfactuals=top_n,
    time_limit=time_limit,
    solver_name=solver_name
)

if lice.stats["optimal"]:
    print("A globally optimal solution found!")

A globally optimal solution found!


In [9]:
print("Suggested counterfactual changes:")
for n, val in zip(sample.index, optimized[0] - sample.values):
    if val > 0:
        print(f"  Increase the {n} parameter by {val}")
    elif val < 0:
        print(f"  Decrease the {n} parameter by {-val}")

Suggested counterfactual changes:
  Decrease the RevolvingUtilizationOfUnsecuredLines parameter by 0.00619451400000004
  Decrease the DebtRatio parameter by 0.092052859
  Decrease the MonthlyIncome parameter by 89.56047399999989
  Increase the NumberOfTimes90DaysLate parameter by 3.0


In [10]:
# Clean-up - remove the neural network model
os.remove("tmp_nn.onnx")